In [17]:
!pip install peakutils
# %pip install neurokit2
# from google.colab import drive
# drive.mount('/content/drive')
# base_dir='/content/drive/MyDrive/ucd/'
base_dir='../../../folders/'


[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [18]:
import matplotlib.pyplot as plt
import neurokit2 as nk
import numpy as np
import pandas as pd
import pickle
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [19]:
data_base=[f'ucddb{i:003d}' for i in range(2, 29)]
data_base.remove('ucddb004')
data_base.remove('ucddb016')
print(data_base)

['ucddb002', 'ucddb003', 'ucddb005', 'ucddb006', 'ucddb007', 'ucddb008', 'ucddb009', 'ucddb010', 'ucddb011', 'ucddb012', 'ucddb013', 'ucddb014', 'ucddb015', 'ucddb017', 'ucddb018', 'ucddb019', 'ucddb020', 'ucddb021', 'ucddb022', 'ucddb023', 'ucddb024', 'ucddb025', 'ucddb026', 'ucddb027', 'ucddb028']


In [20]:
import biosppy
from biosppy.signals import ecg
from scipy.stats import kurtosis

def skewness(data):
    n = len(data)
    mean = np.mean(data)
    stddev = np.std(data, ddof=1)  # ddof=1 for sample standard deviation
    skew = (1/n) * np.sum((data - mean)**3) / (stddev**3)
    return skew

def rp_feature(ecg_signal,sampling_rate=128):

    out = ecg.ecg(signal=ecg_signal, sampling_rate=sampling_rate, show=False)
    rpeaks = out['rpeaks']
    rp_signal = ecg_signal[rpeaks]
    # Maximum value
    rp_max = np.max(rp_signal)

    # Minimum value
    rp_min = np.min(rp_signal)

    # Mean value
    rp_mean = np.mean(rp_signal)

    # Median value
    rp_median = np.median(rp_signal)

    # Standard deviation
    rp_std = np.std(rp_signal)

    # Variance
    rp_var = np.var(rp_signal)

    # Peak-to-peak value
    rp_peak_to_peak = rp_max - rp_min
    
    # Root Mean Square Error
    rp_rmse = np.sqrt(np.sum((rp_signal[1:] - rp_signal[:-1])**2) / (len(rp_signal) - 1))

    # Kurtosis
    rp_kurtosis = kurtosis(rp_signal)

    # Skewness
    rp_skewness = skewness(rp_signal)
   
    # Waveform Factor
    rp_waveform_factor = rp_rmse / rp_mean

    # Peak Factor
    rp_peak_factor = rp_peak_to_peak / rp_rmse

    # Impulse Factor
    rp_impulse_factor = rp_peak_to_peak / rp_mean

    # Margin Factor
    rp_margin_factor = rp_peak_to_peak / np.sqrt(np.sum(np.sqrt(np.abs(rp_signal))) / len(rp_signal))

    # Root Mean Square
    rp_rms = np.sqrt(np.mean(rp_signal**2))
    
    
    # Create a dictionary to store the features
    rp_features = {
        'max': rp_max,
        'min': rp_min,
        'mean': rp_mean,
        'median': rp_median,
        'std': rp_std,
        'var': rp_var,
        'peak_to_peak': rp_peak_to_peak,
        'rmse': rp_rmse,
        'kurtosis': rp_kurtosis,
        'skewness': rp_skewness,
        'waveform_factor': rp_waveform_factor,
        'peak_factor': rp_peak_factor,
        'impulse_factor': rp_impulse_factor,
        'margin_factor': rp_margin_factor,
        'rms': rp_rms
    }

    # Create a DataFrame from the dictionary
    rp_features_df = pd.DataFrame(rp_features, index=[0])
    return rp_features_df
    

In [21]:
for file in data_base:
    print(file)
    
    with open(base_dir+f'all_ecg_stages/{file}_5min.pkl', 'rb') as f:
        res = pickle.load(f)
        stages = res['stages']
        ecgs = np.array(res['ecg']) 
        ecg_win = np.array(res['ecg_win']) 

    df_hrv = pd.DataFrame()   
  
    for i in range(0,len(stages)):
        ecg_sample=ecgs[i]
        ecg_win_sample=ecg_win[i]
        
        try:
          peaks, info = nk.ecg_peaks(ecg_sample, sampling_rate=128)
          hrv = nk.hrv(peaks, sampling_rate=128)
          rp_feuture= rp_feature(ecg_sample,sampling_rate=128)
          
          peaks, info = nk.ecg_peaks(ecg_win_sample, sampling_rate=128)
          hrv_win = nk.hrv(peaks, sampling_rate=128)
          rp_feuture_win= rp_feature(ecg_win_sample,sampling_rate=128)
        except:
          print(i) 
          continue 
        
       
        hrv['tmp'] = 1
        rp_feuture['tmp'] = 1
        hrv = pd.merge(hrv, rp_feuture, on=['tmp'])
        
        hrv_win['tmp'] = 1
        rp_feuture_win['tmp'] = 1
        hrv_win = pd.merge(hrv_win, rp_feuture_win, on=['tmp'])
        
        hrv = pd.merge(hrv, hrv_win, on=['tmp'])
        
        hrv = hrv.drop('tmp', axis=1)
        
        
        hrv['anns']=stages[i]
        df_hrv = pd.concat([df_hrv, hrv], ignore_index=True)
        
    df_hrv.dropna(how='all', axis=1,inplace=True)
    df_hrv.to_csv(base_dir+f'feature/{file}_win.csv', index=False)
  

ucddb002
ucddb003
ucddb005
388
389
391

Intel MKL ERROR: Parameter 6 was incorrect on entry to DGELSD.
392
393
394
395
396
397
ucddb006
319
320
321
322
323
324
325
326
327
328
329
330

Intel MKL ERROR: Parameter 6 was incorrect on entry to DGELSD.
582

Intel MKL ERROR: Parameter 6 was incorrect on entry to DGELSD.
583
584
585
586
587
588
589
590
591
ucddb007
ucddb008
258
259
260
261
262
263
264
265
266
267
268
429
430
431
432
433
434
435
436
437
438
439
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
ucddb009
ucddb010
ucddb011
ucddb012
ucddb013
71

Intel MKL ERROR: Parameter 6 was incorrect on entry to DGELSD.
72
73
75
76
77
78
79
80
81
82
83
84
85
86
87
88

Intel MKL ERROR: Parameter 6 was incorrect on entry to DGELSD.
89
90
91
92
93
94
95
96
97
98
99
100
101
102
106

Intel MKL ERROR: Parameter 6 was incorrect on entry to DGELSD.
ucddb014
ucddb015
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
ucddb017
ucddb018
ucddb019
ucddb020
ucddb021
716
717
718
71

In [22]:
df_hrv.head()

,HRV_MeanNN_x,HRV_SDNN_x,HRV_RMSSD_x,HRV_SDSD_x,HRV_CVNN_x,HRV_CVSD_x,HRV_MedianNN_x,HRV_MadNN_x,HRV_MCVNN_x,HRV_IQRNN_x,...,peak_to_peak_y,rmse_y,kurtosis_y,skewness_y,waveform_factor_y,peak_factor_y,impulse_factor_y,margin_factor_y,rms_y,anns
0,845.128676,61.107553,44.527719,44.285627,0.072306,0.052688,855.46875,40.539844,0.047389,60.546875,...,0.279365,0.111637,-1.326638,0.349545,0.036460,2.502438,0.091239,0.211200,3.063014,0
1,772.381757,70.725115,105.009610,106.490995,0.091568,0.135956,781.25000,23.165625,0.029652,31.250000,...,0.605617,0.116409,-0.022090,0.051335,0.038032,5.202475,0.197862,0.457889,3.062005,0
2,687.872024,76.194760,14.893310,14.798048,0.110769,0.021651,664.06250,92.662500,0.139539,146.484375,...,0.584127,0.117921,-0.012624,0.072767,0.038547,4.953531,0.190944,0.441701,3.060344,0
3,751.602564,112.419155,87.557620,88.713127,0.149573,0.116495,734.37500,104.245312,0.141951,187.500000,...,0.582173,0.116155,0.103103,0.067120,0.037987,5.012058,0.190391,0.440272,3.058951,0
4,772.592905,74.042274,13.085776,12.077030,0.095836,0.016937,781.25000,92.662500,0.118608,132.812500,...,0.582173,0.114179,0.245055,0.047236,0.037379,5.098771,0.190588,0.440386,3.055722,0
